In [1]:
import numpy as np
import numba as nb

In [2]:
levi= np.random.normal(size=(4,4,4,4))
Riemann = np.random.normal(size=(4, 4, 4, 4))
dRuluu = np.random.normal(size=(4, 4, 4, 4))
# levi_mixed = np.random.normal(size=(4, 4, 4, 4))

In [3]:
gk= np.random.normal(size=(4,4))
gkinv= np.linalg.inv(gk)

In [4]:
levi_mixed = np.einsum("px,sy,xyuv->psuv", gk, gk, levi)

In [5]:
dRuluu = 0.5 * np.einsum("abps,psuv->abuv", Riemann, levi)  #!R*^a_{buv}
dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)

In [6]:
%%timeit
dRlluu = np.einsum("ia,abuv->ibuv", gk, dRuluu)

2.93 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
%%timeit
dRlluu =gk.dot(dRuluu.reshape((4, -1))).reshape((4, 4, 4, 4))

1.11 µs ± 78.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
dgk= np.random.normal(size=(4,4,4))

In [9]:
dg = np.einsum("gd,gda->a", gkinv, dgk)  #! D(Det(g))

In [10]:
%%timeit
dg = np.einsum("gd,gda->a", gkinv, dgk)  #! D(Det(g))

1.77 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
dgk.transpose(2, 1, 0).dot(gkinv)

array([[[-4.79967125e+01, -1.36363659e+01,  1.57906350e+01,
          2.25370766e+01],
        [ 2.58684193e+01,  7.00758577e+00, -8.15886991e+00,
         -1.13932053e+01],
        [-4.25342095e+02, -1.07351106e+02,  1.51371781e+02,
          1.84842002e+02],
        [-8.50049564e+01, -2.12784304e+01,  3.11884487e+01,
          3.63158052e+01]],

       [[-2.46221660e+00,  3.26129442e-01,  1.34973789e+00,
          1.20976384e-01],
        [-1.96474604e+02, -4.91384398e+01,  7.06481289e+01,
          8.58922508e+01],
        [ 2.60235199e+02,  6.65358057e+01, -9.18951618e+01,
         -1.14078656e+02],
        [ 2.77769388e+02,  6.70937786e+01, -1.01814747e+02,
         -1.18266572e+02]],

       [[-1.63447885e+02, -4.06881713e+01,  5.89941880e+01,
          7.14595772e+01],
        [-8.51859620e+01, -2.34097719e+01,  2.75368113e+01,
          3.79256745e+01],
        [ 1.31577644e+02,  3.18198796e+01, -4.93597892e+01,
         -5.57941068e+01],
        [-5.26670550e+01, -1.22975561e+

In [12]:
dgk.transpose(2,1,0).dot(gkinv)

array([[[-4.79967125e+01, -1.36363659e+01,  1.57906350e+01,
          2.25370766e+01],
        [ 2.58684193e+01,  7.00758577e+00, -8.15886991e+00,
         -1.13932053e+01],
        [-4.25342095e+02, -1.07351106e+02,  1.51371781e+02,
          1.84842002e+02],
        [-8.50049564e+01, -2.12784304e+01,  3.11884487e+01,
          3.63158052e+01]],

       [[-2.46221660e+00,  3.26129442e-01,  1.34973789e+00,
          1.20976384e-01],
        [-1.96474604e+02, -4.91384398e+01,  7.06481289e+01,
          8.58922508e+01],
        [ 2.60235199e+02,  6.65358057e+01, -9.18951618e+01,
         -1.14078656e+02],
        [ 2.77769388e+02,  6.70937786e+01, -1.01814747e+02,
         -1.18266572e+02]],

       [[-1.63447885e+02, -4.06881713e+01,  5.89941880e+01,
          7.14595772e+01],
        [-8.51859620e+01, -2.34097719e+01,  2.75368113e+01,
          3.79256745e+01],
        [ 1.31577644e+02,  3.18198796e+01, -4.93597892e+01,
         -5.57941068e+01],
        [-5.26670550e+01, -1.22975561e+

In [13]:
np.diag_indices_from(dgk)

(array([0, 1, 2, 3]), array([0, 1, 2, 3]), array([0, 1, 2, 3]))

In [14]:
np.matmul(gk,dgk.transpose(1,0,2)).diagonal(axis1=0,axis2=1).sum(axis=1) -dg

array([-147.52482642,  260.27253761,  208.91867548,  -81.88832521])

In [15]:
np.matmul(gk.reshape((1, -1)),(dgk.reshape((-1, 4)))) -dg

array([[-152.7562776 ,  258.43641671,  217.30005786,  -88.70333191]])

In [16]:
dlevi = -np.einsum("uvps,a->uvpsa", levi, dg) * 0.5

In [17]:
%%timeit
dlevi = -np.einsum("uvps,a->uvpsa", levi, dg) * 0.5


4.88 µs ± 21.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [18]:
(-(np.outer(levi, dg) * 0.5).reshape((4,4,4,4,4)) -dlevi).max()

0.0

In [48]:
%%timeit
dlevi = -(np.outer(levi, dg) * 0.5).reshape((4, 4, 4, 4, 4))

7.4 µs ± 42.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [19]:
dRiemann= np.random.normal(size=(4,4,4,4,4))

In [20]:
DdRuluu = 0.5 * (
    np.einsum("abpsk,psuv->abuvk", dRiemann, levi)
    + np.einsum("abps,psuvk->abuvk", Riemann, dlevi)
)  #! DR*^a_{buv}

In [21]:
%%timeit
np.einsum("ai,ibupk->abupk", gk, DdRuluu)

3.05 µs ± 11 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
%%timeit
gk.dot(DdRuluu)

8.63 µs ± 48.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [23]:
(np.einsum("ai,ibupk->abupk", gk, DdRuluu) - (gk.dot(DdRuluu.reshape((4,-1)))).reshape((4,4,4,4,4))).max()

2.2737367544323206e-13

In [24]:
%%timeit
gk.dot(DdRuluu.reshape((4,-1))).reshape((4,4,4,4,4))

1.61 µs ± 16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
dgkinv= np.random.normal(size=(4,4,4))

In [26]:
%%timeit
DddRuuuu = 0.5 * (
    np.einsum("bi,aipsk,psuv->abuvk", gkinv, DdRuluu, levi_mixed, optimize=True)
    + np.einsum("bik,aips,psuv->abuvk", dgkinv, dRuluu, levi_mixed, optimize=True)
    + np.einsum(
        "bi,aips,pm,sn,mnuvz->abuvz", gkinv, dRuluu, gk, gk, dlevi, optimize=True
    )
    + np.einsum(
        "bi,aips,pmz,sn,mnuv->abuvz", gkinv, dRuluu, dgk, gk, levi, optimize=True
    )
    + np.einsum(
        "bi,aips,pm,snz,mnuv->abuvz", gkinv, dRuluu, gk, dgk, levi, optimize=True
    )
)  #! D*R*^{abuv}

657 µs ± 7.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
%%timeit
DddRuuuu = 0.5 * (
    np.einsum("bi,aipsk,psuv->abuvk", gkinv, DdRuluu, levi_mixed)
    + np.einsum("bik,aips,psuv->abuvk", dgkinv, dRuluu, levi_mixed)
    + np.einsum(
        "bi,aips,pm,sn,mnuvz->abuvz", gkinv, dRuluu, gk, gk, dlevi,
    )
    + np.einsum(
        "bi,aips,pmz,sn,mnuv->abuvz", gkinv, dRuluu, dgk, gk, levi,
    )
    + np.einsum(
        "bi,aips,pm,snz,mnuv->abuvz", gkinv, dRuluu, gk, dgk, levi,
    )
)  #! D*R*^{abuv}

19.1 ms ± 401 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
ddRuuuu= np.random.normal(size=(4,4,4,4))
pvector= np.random.normal(size=(4))
svector = np.random.normal(size=(4))

In [29]:
wu = -np.einsum("uabg,a,b,g->u", ddRuuuu, svector, pvector, svector)  #!w^mu

In [30]:
%%timeit
wu = -np.einsum("uabg,a,b,g->u", ddRuuuu, svector, pvector, svector)  #!w^mu

4.82 µs ± 407 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
%%timeit
wu= -ddRuuuu.dot(svector).dot(pvector).dot(svector)

2.56 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [32]:
(-np.einsum("uabg,a,b,g->u", ddRuuuu, svector, pvector, svector)) - (
    -ddRuuuu.dot(svector).dot(pvector).dot(svector)
)

array([ 8.88178420e-16,  0.00000000e+00, -3.33066907e-16,  0.00000000e+00])

In [33]:
DddRuuuu= np.random.normal(size=(4,4,4,4,4))

In [34]:
wuv = -np.einsum("uabgv,a,b,g->uv", DddRuuuu, svector, pvector, svector)

In [35]:
wuv

array([[-4.97509537,  2.72647315,  2.4847986 , -1.20860172],
       [ 2.92344023, -6.11625078, -0.0913571 ,  5.1735743 ],
       [ 1.58374699,  4.04820965,  1.62312056,  1.04479312],
       [-3.21127473,  0.86785244, -2.53032938,  1.03578971]])

In [36]:
%%timeit
wuv = -np.einsum("uabgv,a,b,g->uv", DddRuuuu, svector, pvector, svector)

11.6 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [79]:
%%timeit
wuv= -DddRuuuu.transpose(0,4,1,2,3).dot(svector).dot(pvector).dot(svector)

6.13 µs ± 377 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [80]:
-DddRuuuu.transpose(0,4,1,2,3).dot(svector).dot(pvector).dot(svector) -wuv

array([[ 0.00000000e+00,  0.00000000e+00, -1.33226763e-15,
        -4.44089210e-16],
       [ 1.11022302e-15, -4.44089210e-16, -2.91433544e-16,
         8.32667268e-17],
       [ 4.44089210e-16,  4.44089210e-16, -4.44089210e-16,
        -1.11022302e-16],
       [ 1.90819582e-16, -4.44089210e-16, -1.11022302e-16,
         2.22044605e-16]])

In [81]:
uvector= np.random.normal(size=4)

In [83]:
%%timeit
uvectorl= np.einsum("ai,i->a",gk,uvector)

1.54 µs ± 15.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [84]:
%%timeit
uvectorl= gk.dot(uvector)

316 ns ± 8.68 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [86]:
uvectorl= np.einsum("ai,i->a", gk, uvector) 

In [88]:
%%timeit
np.einsum("a,auv->uv", pvector, dgkinv)

1.71 µs ± 28.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [93]:
%%timeit
dgkinv.transpose(1,2,0).dot(pvector)

745 ns ± 6.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [92]:
np.einsum("a,auv->uv", pvector, dgkinv) - dgkinv.transpose(1,2,0).dot(pvector)

array([[ 5.55111512e-17,  5.55111512e-17, -2.22044605e-16,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.22044605e-16],
       [ 0.00000000e+00,  0.00000000e+00,  2.77555756e-17,
         4.44089210e-16]])

In [97]:
%%timeit
np.einsum("u,a,av->uv", uvector, uvectorl, wuv)

2.33 µs ± 50.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [98]:
%%timeit
np.matmul(np.outer(uvector,uvectorl),wuv)

2.77 µs ± 25.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [106]:
@nb.njit
def outer_numba(a, b):
    m = a.shape[0]
    n = b.shape[0]
    result = np.empty((m, n))
    for i in range(m):
        for j in range(n):
            result[i, j] = a[i] * b[j]
    return result

In [107]:
outer_numba(uvector, uvectorl)

array([[ 0.51289035, -0.17992221, -0.33154933, -0.08855024],
       [-6.28213996,  2.20377809,  4.06098353,  1.08460805],
       [11.04393841, -3.87421956, -7.13916789, -1.90672997],
       [-7.15871761,  2.51128201,  4.62763238,  1.23594871]])

In [108]:
np.einsum("u,a,av->uv", uvector, uvectorl, wuv) - (
    np.matmul(outer_numba(uvector, uvectorl), wuv)
)

array([[ 0.00000000e+00, -2.22044605e-16,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 3.55271368e-15,  3.55271368e-15, -7.10542736e-15,
         0.00000000e+00]])

In [109]:
%%timeit
np.matmul(outer_numba(uvector,uvectorl),wuv)

1.81 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [110]:
%%timeit
np.einsum("u,a,b,abv->uv", uvector, wu, wu, dgk)

4.88 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [116]:
%%timeit
np.matmul(
    np.outer(uvector, outer_numba(wu, wu)).reshape((4, -1)), dgk.reshape((-1, 4))
)

4.09 µs ± 50 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [128]:
%%timeit
outer_numba(uvector,wu.dot(wu.dot(dgk)))

1.6 µs ± 30.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [129]:
outer_numba(uvector, wu.dot(wu.dot(dgk))) - (
    np.einsum("u,a,b,abv->uv", uvector, wu, wu, dgk)
)

array([[-3.49113100e-17,  1.04083409e-16,  0.00000000e+00,
        -7.63278329e-17],
       [ 6.24500451e-17,  2.91433544e-16, -6.66133815e-16,
         1.05471187e-15],
       [ 9.92261828e-16,  3.60822483e-16,  3.10862447e-15,
        -1.94289029e-15],
       [ 4.51028104e-17,  1.05471187e-15,  2.22044605e-16,
         1.41553436e-15]])

In [115]:
np.matmul(
    np.outer(uvector, outer_numba(wu, wu)).reshape((4, -1)), dgk.reshape((-1, 4))
) - (np.einsum("u,a,b,abv->uv", uvector, wu, wu, dgk))

array([[-1.40946282e-17,  7.45931095e-17,  0.00000000e+00,
        -2.25514052e-17],
       [ 3.81639165e-17, -6.93889390e-17, -1.33226763e-15,
         1.66533454e-16],
       [ 9.36750677e-16,  1.30451205e-15,  3.55271368e-15,
        -1.33226763e-15],
       [-3.46944695e-17,  8.32667268e-17, -2.22044605e-16,
         8.32667268e-17]])

In [117]:
N=1.045

In [134]:
%%timeit
dx_dx = N * (
        dgkinv.transpose(1, 2, 0).dot(pvector)
        + wuv
        + np.matmul(outer_numba(uvector, uvectorl), wuv)
        + N / 2 * outer_numba(uvector, wu.dot(wu.dot(dgk)))
    )

7.55 µs ± 52.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [131]:
%%timeit
dx_dx= N*(np.einsum("a,auv->uv",pvector,dgkinv) + wuv + np.einsum("u,a,av->uv",uvector,uvectorl,wuv) +N/2 *(np.einsum("u,a,b,abv->uv",uvector,wu,wu,dgk)))

12.4 µs ± 208 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [132]:
(
    N
    * (
        dgkinv.transpose(1, 2, 0).dot(pvector)
        + wuv
        + np.matmul(np.outer(uvector, uvectorl), wuv)
        + N / 2 * outer_numba(uvector, wu.dot(wu.dot(dgk)))
    )
) - (
    N
    * (
        np.einsum("a,auv->uv", pvector, dgkinv)
        + wuv
        + np.einsum("u,a,av->uv", uvector, uvectorl, wuv)
        + N / 2 * (np.einsum("u,a,b,abv->uv", uvector, wu, wu, dgk))
    )
)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         8.88178420e-16],
       [ 0.00000000e+00,  0.00000000e+00,  7.10542736e-15,
         0.00000000e+00],
       [-3.55271368e-15, -7.10542736e-15,  7.10542736e-15,
         0.00000000e+00]])

In [140]:
Wuv = -np.einsum("uavb,a,b->uv", ddRuuuu, svector, svector)

In [141]:
%%timeit
Wuv = -np.einsum("uavb,a,b->uv", ddRuuuu, svector, svector) 

3.8 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [142]:
%%timeit
Wuv= -ddRuuuu.transpose(0,2,1,3).dot(svector).dot(svector)

2.24 µs ± 25.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [143]:
-ddRuuuu.transpose(0,2,1,3).dot(svector).dot(svector) -Wuv

array([[ 0.00000000e+00, -1.11022302e-16,  5.55111512e-17,
        -2.22044605e-16],
       [ 0.00000000e+00, -1.11022302e-16,  1.11022302e-16,
        -1.38777878e-17],
       [ 0.00000000e+00,  1.11022302e-16, -1.11022302e-16,
        -2.22044605e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00]])

In [144]:
%%timeit
np.einsum("u,a,av->uv",uvector,uvectorl,Wuv)

2.29 µs ± 28.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [145]:
%%timeit
outer_numba(uvector,uvectorl.dot(Wuv))

1.02 µs ± 17.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [146]:
np.einsum("u,a,av->uv", uvector, uvectorl, Wuv) - (
    outer_numba(uvector, uvectorl.dot(Wuv))
)

array([[ 6.07153217e-17,  0.00000000e+00,  4.16333634e-17,
         5.55111512e-17],
       [-3.33066907e-16, -8.88178420e-16, -1.11022302e-16,
        -4.44089210e-16],
       [ 4.16333634e-16,  0.00000000e+00,  0.00000000e+00,
         6.66133815e-16],
       [ 3.05311332e-16, -8.88178420e-16,  1.11022302e-16,
        -4.44089210e-16]])

In [151]:
%%timeit
dx_dP = N * (
        gkinv
        + Wuv
        + N * outer_numba(uvector, wu)
        + outer_numba(uvector, uvectorl.dot(Wuv))
    )


4.6 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [150]:
%%timeit
dx_dP= N*(gkinv + Wuv + N*outer_numba(uvector,wu) + np.einsum("u,a,av->uv",uvector,uvectorl,Wuv))

6.09 µs ± 63.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [149]:
N * (
    gkinv + Wuv + N * outer_numba(uvector, wu) + outer_numba(uvector, uvectorl.dot(Wuv))
) - (
    N
    * (
        gkinv
        + Wuv
        + N * outer_numba(uvector, wu)
        + np.einsum("u,a,av->uv", uvector, uvectorl, Wuv)
    )
)

array([[-5.55111512e-17,  0.00000000e+00, -5.55111512e-17,
         0.00000000e+00],
       [ 4.44089210e-16,  8.88178420e-16,  1.11022302e-16,
         6.66133815e-16],
       [-2.22044605e-16,  0.00000000e+00,  0.00000000e+00,
        -8.88178420e-16],
       [-4.44089210e-16,  8.88178420e-16, -4.44089210e-16,
         4.44089210e-16]])

In [152]:
Vuv = -(
    ((ddRuuuu.transpose(0, 1, 3, 2)).dot(pvector).transpose(0, 2, 1)).dot(svector)
    - ddRuuuu.dot(pvector).dot(svector)
)  #! V^{uv}

In [156]:
%%timeit
np.einsum("u,a,av->uv", uvector, uvectorl, Vuv)

2.25 µs ± 6.16 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [157]:
%%timeit
outer_numba(uvector, uvectorl.dot(Vuv)) 

1.01 µs ± 7.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [155]:
outer_numba(uvector, uvectorl.dot(Vuv)) - (
    np.einsum("u,a,av->uv", uvector, uvectorl, Vuv)
)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -7.63278329e-17],
       [ 0.00000000e+00,  8.88178420e-16,  0.00000000e+00,
         2.27595720e-15],
       [-3.55271368e-15,  1.77635684e-15, -7.10542736e-15,
        -8.88178420e-16],
       [-1.77635684e-15, -1.77635684e-15, -3.55271368e-15,
        -8.88178420e-16]])

In [158]:
term1 = -np.einsum("b,ugvb,g->uv", svector, dRlluu, uvector)

In [159]:
%%timeit
term1 = -np.einsum("b,ugvb,g->uv", svector, dRlluu, uvector)

3.68 µs ± 27.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [160]:
%%timeit
term1= -dRlluu.transpose(0,2,1,3).dot(svector).dot(uvector)

2.3 µs ± 52.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [161]:
(-np.einsum("b,ugvb,g->uv", svector, dRlluu, uvector)) - (
    -dRlluu.transpose(0, 2, 1, 3).dot(svector).dot(uvector)
)

array([[ 1.77635684e-15, -2.22044605e-16, -8.88178420e-16,
         8.88178420e-16],
       [ 4.44089210e-16,  0.00000000e+00, -1.77635684e-15,
         3.55271368e-15],
       [ 1.77635684e-15, -2.22044605e-16, -8.88178420e-16,
         0.00000000e+00],
       [ 0.00000000e+00, -8.88178420e-16, -8.88178420e-16,
         0.00000000e+00]])

In [162]:
term1 = -np.einsum("ugav,g,a-> uv", dRlluu, uvector, pvector)

In [163]:
%%timeit
term1 = -np.einsum("ugav,g,a-> uv", dRlluu, uvector, pvector)

3.89 µs ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [165]:
-dRlluu.transpose(0,3,1,2).dot(pvector).dot(uvector) -(term1)

array([[-1.77635684e-15, -1.77635684e-15,  3.55271368e-15,
        -3.55271368e-15],
       [ 0.00000000e+00, -1.77635684e-15, -3.55271368e-15,
        -1.42108547e-14],
       [-1.77635684e-15, -3.55271368e-15,  0.00000000e+00,
         1.77635684e-15],
       [ 3.55271368e-15,  3.55271368e-15,  3.55271368e-15,
         0.00000000e+00]])

In [166]:
%%timeit
term1= -dRlluu.transpose(0,3,1,2).dot(pvector).dot(uvector) 

2.33 µs ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [167]:
dx_dx= np.random.normal(size=(4,4))

In [168]:
term2 = -np.outer(
    pvector,
    dRuluu.transpose(1, 0, 2, 3).dot(svector).dot(pvector).dot(svector).dot(dx_dx),
)

In [169]:
%%timeit
term2 = -np.outer(
        pvector,
        dRuluu.transpose(1, 0, 2, 3).dot(svector).dot(pvector).dot(svector).dot(dx_dx),
    )

4.83 µs ± 89.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [170]:
%%timeit

term2 = -outer_numba(
        pvector,
        dRuluu.transpose(1, 0, 2, 3).dot(svector).dot(pvector).dot(svector).dot(dx_dx),
    )


3.65 µs ± 27.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [171]:
%%timeit
term2 = -np.outer(
        pvector, dRuluu.transpose(2, 0, 1, 3).dot(svector).dot(uvector).dot(svector)
    )

4.59 µs ± 92.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [172]:
%%timeit
term2 = -outer_numba(
        pvector, dRuluu.transpose(2, 0, 1, 3).dot(svector).dot(uvector).dot(svector)
    )

3.53 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [173]:
np.einsum("ij,j,i->", gkinv, pvector, svector)

1.0073413112597858

In [175]:
gkinv.dot(pvector).dot(svector) - np.einsum("ij,j,i->", gkinv, pvector, svector)

-2.220446049250313e-16

In [176]:
Stensor= np.random.normal(size=(4,4))


In [178]:
dgkt= np.random.normal(size=(4,4))

In [185]:
%%timeit
np.einsum("uv,uv->", dgkt, Stensor)

1.63 µs ± 61.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [184]:
%%timeit
dgkt.dot(Stensor.transpose()).diagonal().sum()

1.63 µs ± 26.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [183]:
dgkt.dot(Stensor.transpose()).diagonal().sum() - np.einsum("uv,uv->", dgkt, Stensor)

-1.5543122344752192e-15

In [37]:
%%timeit
dRuluu= 0.5*np.einsum("abps,psuv->abuv",Riemann,levi)

4.55 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [38]:
%%timeit
dRuluu= 0.5*np.matmul(Riemann.reshape((-1,16)),levi.reshape((16,-1))).reshape((4,4,4,4))

2.85 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [40]:
(0.5 * np.einsum("abps,psuv->abuv", Riemann, levi) - (
    0.5
    * np.matmul(Riemann.reshape((-1, 16)), levi.reshape((16, -1))).reshape((4, 4, 4, 4))
)).max()

8.881784197001252e-16

In [41]:
%%timeit
levi_mixed= np.einsum("px,sy,xyuv->psuv",gk,gk,levi)

9.96 µs ± 369 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [42]:
%%timeit
levi_mixed = (
        gk.dot(
            gk.dot(levi.reshape((4, -1)))
            .reshape((4, 4, 4, 4))
            .transpose(1, 0, 2, 3)
            .reshape((4, -1))
        )
        .reshape((4, 4, 4, 4))
        .transpose(1, 0, 2, 3)
    )

2.79 µs ± 92.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [44]:
((
    gk.dot(
        gk.dot(levi.reshape((4, -1)))
        .reshape((4, 4, 4, 4))
        .transpose(1, 0, 2, 3)
        .reshape((4, -1))
    )
    .reshape((4, 4, 4, 4))
    .transpose(1, 0, 2, 3)
) - (np.einsum("px,sy,xyuv->psuv", gk, gk, levi))).max()

1.7763568394002505e-15

In [46]:
%%timeit
dRlluu = gk.dot(dRuluu.reshape((4, -1))).reshape((4, 4, 4, 4))

948 ns ± 20.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [47]:
%%timeit
dRlluu= np.einsum("ia,abuv->ibuv",gk,dRuluu)

2.24 µs ± 8.89 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [49]:
(
    (gk.dot(dRuluu.reshape((4, -1))).reshape((4, 4, 4, 4)))
    - (np.einsum("ia,abuv->ibuv", gk, dRuluu))
).max()

1.7763568394002505e-15

In [51]:
%%timeit
dlevi = -np.einsum("uvps,a->uvpsa", levi, dg) * 0.5

4.96 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [58]:
%%timeit
dlevi= -np.outer(levi,dg).reshape((4,4,4,4,4)) * 0.5

10.8 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [56]:
np.outer(levi, dg)

(256, 4)

In [57]:
-np.einsum("uvps,a->uvpsa", levi, dg) * 0.5 - (-np.outer(levi,dg).reshape((4,4,4,4,4)) * 0.5)

array([[[[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.],
